In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import re
import string

from tensorflow.keras import layers
from tensorflow.keras import losses

# requires update to tensorflow 2.4
# >>> conda activate PIC16B
# >>> pip install tensorflow==2.4
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.layers.experimental.preprocessing import StringLookup

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/benbrill/MoodSpace/main/trainingSongs_clean.csv")
df

,Unnamed: 0,trackName,artist,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,lyrics,cluster,language
0,3,Overthinker,INZO,4K9xid96G3YmIvQZXN9SXg,0.472,0.605,8.0,-4.437,1.0,0.1340,0.03110,0.030800,0.1010,0.212,128.375,audio_features,a person who thinks all the time has nothing t...,0,en
1,4,Lifestyles of the Rich & Famous,Good Charlotte,2g2a5kDeZexbUTD8abcvm6,0.620,0.930,1.0,-3.685,1.0,0.0374,0.00043,0.000000,0.0686,0.609,106.220,audio_features,always see it on t v or read in the magazines ...,3,en
2,6,Carrying Your Love With Me,George Strait,7puxIVNdj5nsBJk43zM3bH,0.629,0.479,10.0,-10.608,1.0,0.0271,0.22000,0.000000,0.0587,0.345,138.231,audio_features,baby all i got is this beat up leather bag and...,5,en
3,7,"Check Yes, Juliet",We The Kings,0wVluBsVAVzBKrqspuCcwR,0.352,0.912,7.0,-4.253,1.0,0.0725,0.00197,0.000000,0.1930,0.351,166.795,audio_features,check yes juliet are you with me rain is falli...,2,en
4,8,At My Worst (feat. Kehlani),Pink Sweat$,58w68w4s8h9gw3xrDaXyuj,0.731,0.484,0.0,-5.579,1.0,0.0354,0.73000,0.000003,0.3260,0.350,92.043,audio_features,can i call you baby can you be my friend can y...,1,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
712,994,90mh,Trefuego,1VxvGm1moDJ3svQlwjdBwA,0.716,0.427,1.0,-8.993,1.0,0.0529,0.18700,0.000000,0.3540,0.223,108.993,audio_features,hi kevin ayy yeah ayy ayy you dont really want...,3,en
713,995,9 Bridge,Rowdy Rebel,2sHekv6OdEiO4htSjdB9j4,0.642,0.589,5.0,-7.392,0.0,0.3020,0.12400,0.000000,0.0868,0.720,95.543,audio_features,i know i dont never make promises this time i ...,1,en
714,996,Lotus Flower Bomb (feat. Miguel),Wale,3MAgQuClHcAV8E9CbeBS6f,0.512,0.598,9.0,-4.959,0.0,0.1150,0.61100,0.000000,0.0881,0.345,70.189,audio_features,ima rap to you real quick i wanna enjoy the lu...,4,en
715,997,Envy Me,Calboy,7rvyVWja33WG9R97oeJAjx,0.740,0.488,1.0,-7.664,0.0,0.2700,0.23400,0.000000,0.2410,0.584,149.042,audio_features,now i lay me down to sleep now i lay me down t...,5,en


In [3]:
# only the top distinct words will be tracked
max_tokens = 2000

# each headline will be a vector of length 25
sequence_length = 500

vectorize_layer = TextVectorization(
    max_tokens=max_tokens, # only consider this many words
    output_mode='int',
    output_sequence_length=sequence_length) 

In [4]:
data = tf.data.Dataset.from_tensor_slices((df["lyrics"], df["cluster"]))

In [5]:
data = data.shuffle(buffer_size = len(data))

train_size = int(0.7*len(data))
val_size   = int(0.1*len(data))

train = data.take(train_size)
val   = data.skip(train_size).take(val_size)
test  = data.skip(train_size + val_size)
len(train), len(val), len(test)

(501, 71, 145)

In [9]:
def vectorize_headline(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), [label]

train_vec = train.map(vectorize_headline)
val_vec   = val.map(vectorize_headline)
test_vec  = test.map(vectorize_headline)

In [6]:
model = tf.keras.Sequential([
  layers.Embedding(max_tokens, output_dim = 3, name="embedding"),
  layers.Dropout(0.2),
  layers.GlobalAveragePooling1D(),
  layers.Dropout(0.2),
  layers.Dense(8)]
)

In [7]:
model.compile(loss=losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer='adam', 
              metrics=['accuracy'])

In [12]:
history = model.fit(train_vec, epochs = 100, validation_data = val_vec)

Epoch 1/50
501/501 [==============================] - 1s 3ms/step - loss: 1.8731 - accuracy: 0.2455 - val_loss: 1.8028 - val_accuracy: 0.2394
Epoch 2/50
501/501 [==============================] - 1s 3ms/step - loss: 1.8616 - accuracy: 0.2515 - val_loss: 1.7851 - val_accuracy: 0.3239
Epoch 3/50
501/501 [==============================] - 1s 3ms/step - loss: 1.8372 - accuracy: 0.2794 - val_loss: 1.8630 - val_accuracy: 0.2394
Epoch 4/50
501/501 [==============================] - 1s 3ms/step - loss: 1.8464 - accuracy: 0.2475 - val_loss: 1.8881 - val_accuracy: 0.2817
Epoch 5/50
501/501 [==============================] - 1s 3ms/step - loss: 1.8540 - accuracy: 0.2555 - val_loss: 1.8053 - val_accuracy: 0.2958
Epoch 6/50
501/501 [==============================] - 1s 3ms/step - loss: 1.8562 - accuracy: 0.2475 - val_loss: 1.8210 - val_accuracy: 0.2254
Epoch 7/50
501/501 [==============================] - 1s 3ms/step - loss: 1.8434 - accuracy: 0.2655 - val_loss: 1.8416 - val_accuracy: 0.2958
Epoch 

In [19]:
model.predict(test_vec)[1]

array([ 0.301615  ,  0.61373496, -0.37419444, -0.2880766 ,  0.04941263,
        0.26342294, -0.91452265, -4.326766  ], dtype=float32)

In [17]:
test_vec

<MapDataset shapes: ((None, 500), (1,)), types: (tf.int64, tf.int64)>